# Get top Video Games through Gamespot API


### ABSTRACT


We are working on a video game ratings dataset and will be extracting video game data from 3 different data sources and then will be munging them together to form a consistent dataset. 

## Data

Data gathered from Gamespot API mentioned below. Data consists of 








## Importing Libraries

In [1]:
import requests
import pandas as pd
import json
import os
import rawgpy
import http.client

### DATA SOURCE 1 : USING API

### Importing the dataset using API

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [3]:
# Fetching the tags from the website
url = 'http://www.vgchartz.com/gamedb/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=Yes&results=200&order=Sales&showtotalsales=0&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&showdeveloper=0&showcriticscore=0&showpalsales=0&showreleasedate=0&showuserscore=0&showjapansales=0&showlastupdate=0&showothersales=0&showshipped=0'
html = requests.get(url, headers = {'User-Agent':'Mozilla/5.0'})
print(html.status_code)


200


In [107]:
soup = BeautifulSoup(html.content, 'html.parser')
k=[]
rank=[]
gname=[]
publisher=[]
platform=[]
release_date=[]
for tag in soup.find_all('a'):
    if tag['href'].startswith('http://www.vgchartz.com/game/'):
        k.append(tag.get_text().strip())
        data=tag.parent.parent.find_all('td')
        if data!=[]:
            rank.append(np.int32(data[0].string))
            platform.append(data[3].find('img').attrs['alt'].strip(' '))
            publisher.append(data[4].string.strip(' '))
gname = k[10:]
columns = {
    'Rank': rank,
    'Name':gname,
    'Platform':platform,
    'Publisher':publisher
}
df = pd.DataFrame(columns)
print(df.columns)
df = df[[
    'Rank', 'Name', 'Platform',
    'Publisher']]
df.to_csv("vgsales.csv", sep=",", encoding='utf-8', index=False)

#df.drop_duplicates(subset ="Name",keep = False, inplace = True)
print (df)

Index(['Rank', 'Name', 'Platform', 'Publisher'], dtype='object')
     Rank                                      Name Platform  \
0       1                                Wii Sports      Wii   
1       2                         Super Mario Bros.      NES   
2       3                            Mario Kart Wii      Wii   
3       4             PLAYERUNKNOWN'S BATTLEGROUNDS       PC   
4       5                                 Minecraft       PC   
..    ...                                       ...      ...   
195   196                     Red Dead Redemption 2     XOne   
196   197                                   Destiny      PS4   
197   198                                  Tekken 2       PS   
198   199    Uncharted: The Nathan Drake Collection      PS4   
199   200  Super Mario World: Super Mario Advance 2      GBA   

                       Publisher  
0                       Nintendo  
1                       Nintendo  
2                       Nintendo  
3               PUBG Corpo

In [10]:
import rawgpy
import json
rawg_ratings = []
genre = []
rawg = rawgpy.RAWG("User-Agent, this should identify your app")
for name in gname:
    results = rawg.search(name)  # defaults to returning the top 5 results
    game = results[0]
    game.populate()
    rawg_ratings.append(game.rating)
    rawgpy.
print (len(rawg_ratings))

200


In [11]:
df['RAWG Ratings']=rawg_ratings
df

,Rank,Name,Platform,Publisher,RAWG Ratings
0,1,Wii Sports,Wii,Nintendo,4.20
1,2,Super Mario Bros.,NES,Nintendo,4.27
2,3,Mario Kart Wii,Wii,Nintendo,4.16
3,4,PLAYERUNKNOWN'S BATTLEGROUNDS,PC,PUBG Corporation,3.40
4,5,Minecraft,PC,Mojang,4.33
...,...,...,...,...,...
195,196,Red Dead Redemption 2,XOne,Rockstar Games,4.53
196,197,Destiny,PS4,Activision,3.78
197,198,Tekken 2,PS,Namco,4.07
198,199,Uncharted: The Nathan Drake Collection,PS4,Sony Computer Entertainment,4.40


In [108]:
print(df.isnull().any())
print(df.columns)
dframe = pd.read_csv('vgsales2019.csv')
dframe.drop(['Rank','ESRB_Rating','Platform','Publisher','Developer','User_Score', 'Total_Shipped', 'Global_Sales', 'NA_Sales', 'PAL_Sales', 'JP_Sales', 'Other_Sales'], axis=1, inplace= True)

Rank         False
Name         False
Platform     False
Publisher    False
dtype: bool
Index(['Rank', 'Name', 'Platform', 'Publisher'], dtype='object')


In [109]:
print (dframe.isna().sum())
dframe.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
dframe.head(100)
print(dframe.columns)

Name                0
Genre               0
Critic_Score    49256
Year              979
dtype: int64
Index(['Name', 'Genre', 'Critic_Score', 'Year'], dtype='object')


In [169]:
# Merging the two dataframe df and dframe having unique names
result = pd.merge(df,dframe, 
                 on = 'Name')

In [33]:
result

,Rank,Name,Platform,Publisher,RAWG Ratings,Genre,Critic_Score,Year
0,1,Wii Sports,Wii,Nintendo,4.20,Sports,7.7,2006.0
1,2,Super Mario Bros.,NES,Nintendo,4.27,Platform,10.0,1985.0
2,2,Super Mario Bros.,NES,Nintendo,4.27,Platform,9.0,2006.0
3,3,Mario Kart Wii,Wii,Nintendo,4.16,Racing,8.2,2008.0
4,5,Minecraft,PC,Mojang,4.33,Misc,10.0,2010.0
...,...,...,...,...,...,...,...,...
309,194,Link's Crossbow Training,Wii,Nintendo,3.67,Shooter,6.9,2007.0
310,195,New Super Mario Bros. U,WiiU,Nintendo,3.88,Platform,8.4,2012.0
311,197,Destiny,PS4,Activision,3.78,Shooter,8.3,2014.0
312,198,Tekken 2,PS,Namco,4.07,Fighting,9.1,1996.0


In [111]:
result.to_csv("result.csv",encoding="utf-8",index=False)

In [81]:
#result.drop_duplicates(subset ="Name",keep = "first", inplace = True)
print(result)

     Rank                                      Name Platform   Publisher  \
0       1                                Wii Sports      Wii    Nintendo   
1       2                         Super Mario Bros.      NES    Nintendo   
2       2                         Super Mario Bros.      NES    Nintendo   
3       3                            Mario Kart Wii      Wii    Nintendo   
4       5                                 Minecraft       PC      Mojang   
..    ...                                       ...      ...         ...   
309   194                  Link's Crossbow Training      Wii    Nintendo   
310   195                   New Super Mario Bros. U     WiiU    Nintendo   
311   197                                   Destiny      PS4  Activision   
312   198                                  Tekken 2       PS       Namco   
313   200  Super Mario World: Super Mario Advance 2      GBA    Nintendo   

     RAWG Ratings     Genre  Critic_Score    Year  
0            4.20    Sports        

In [126]:

nm=''
gen = {} # dictionary to store the values of different Genres
plat = {} # dictionary to store the values of Platform 
year = {}  # dictionary to store the va;ues of year of release
print(len(result))
for i in range(0,314):
    gen.setdefault(result["Name"][i], {})[result["Genre"][i]] = ''
    #To eliminate the duplication of key-value pairs
    plat.setdefault(result["Name"][i], {})[result["Platform"][i]] = ''  


314


In [160]:
title = list(gen.keys()) #returns a list of Keys(Names of the game)
len(title)
for t in title:
    print (t)

Wii Sports
Super Mario Bros.
Mario Kart Wii
Minecraft
Wii Sports Resort
New Super Mario Bros.
New Super Mario Bros. Wii
Tetris
Wii Play
Kinect Adventures!
Nintendogs
Mario Kart DS
Wii Fit
Wii Fit Plus
Super Mario World
Grand Theft Auto V
Brain Age: Train Your Brain in Minutes a Day
Mario Kart 8 Deluxe
Mario Kart 7
Super Mario Land
Grand Theft Auto: San Andreas
Super Mario Bros. 3
Uncharted 4: A Thief's End
Grand Theft Auto: Vice City
Super Smash Bros. Ultimate
Super Mario Odyssey
Counter-Strike: Source
Sonic the Hedgehog
Gran Turismo 3: A-Spec
Brain Age 2: More Training in Minutes a Day
Call of Duty: Modern Warfare 3
Call of Duty: Black Ops
The Legend of Zelda: Breath of the Wild
Halo 3
Red Dead Redemption 2
Call of Duty: Black Ops II
Call of Duty: Modern Warfare 2
Call of Duty: WWII
Super Smash Bros. Brawl
New Super Mario Bros. 2
Marvel's Spider-Man
Grand Theft Auto III
Portal 2
Super Mario Galaxy
Super Mario 3D Land
Animal Crossing: New Leaf
Diablo III
Gran Turismo 5
Super Mario 64
F

In [157]:
gek = list(gen.values())
genr = []
for k in gek:
    genr.append(list(k.keys()))
len(genr)

145

In [172]:
result.drop_duplicates(subset ="Name",keep = 'first', inplace = True)
for t in result['Name']:
    print (t)
len(result)

Wii Sports
Super Mario Bros.
Mario Kart Wii
Minecraft
Wii Sports Resort
New Super Mario Bros.
New Super Mario Bros. Wii
Tetris
Wii Play
Kinect Adventures!
Nintendogs
Mario Kart DS
Wii Fit
Wii Fit Plus
Super Mario World
Grand Theft Auto V
Brain Age: Train Your Brain in Minutes a Day
Mario Kart 8 Deluxe
Mario Kart 7
Super Mario Land
Grand Theft Auto: San Andreas
Super Mario Bros. 3
Uncharted 4: A Thief's End
Grand Theft Auto: Vice City
Super Smash Bros. Ultimate
Super Mario Odyssey
Counter-Strike: Source
Sonic the Hedgehog
Gran Turismo 3: A-Spec
Brain Age 2: More Training in Minutes a Day
Call of Duty: Modern Warfare 3
Call of Duty: Black Ops
The Legend of Zelda: Breath of the Wild
Halo 3
Red Dead Redemption 2
Call of Duty: Black Ops II
Call of Duty: Modern Warfare 2
Call of Duty: WWII
Super Smash Bros. Brawl
New Super Mario Bros. 2
Marvel's Spider-Man
Grand Theft Auto III
Portal 2
Super Mario Galaxy
Super Mario 3D Land
Animal Crossing: New Leaf
Diablo III
Gran Turismo 5
Super Mario 64
F

145

In [153]:
critic_score = result['Critic_Score']
Id = result['Rank']
year = result['Year']
critic_score



0      7.7
3      8.2
24     8.0
25     9.1
26     8.6
      ... 
309    6.9
310    8.4
311    8.3
312    9.1
313    9.4
Name: Critic_Score, Length: 92, dtype: float64

In [154]:
Id

0        1
3        3
24       6
25       8
26       9
      ... 
309    194
310    195
311    197
312    198
313    200
Name: Rank, Length: 92, dtype: int64

In [155]:
year

0      2006.0
3      2008.0
24     2009.0
25     2006.0
26     2009.0
        ...  
309    2007.0
310    2012.0
311    2014.0
312    1996.0
313    2002.0
Name: Year, Length: 92, dtype: float64